# Importing Header and SoilPrep

In [1]:
import import_ipynb 
from Header import *

importing Jupyter notebook from Header.ipynb


In [2]:
import import_ipynb
from SoilPrep import * 

importing Jupyter notebook from SoilPrep.ipynb


## Loading Data and MetaData

In [3]:
import import_ipynb
from LoadDataMetaData import *

importing Jupyter notebook from LoadDataMetaData.ipynb


## Fixed One Time Resampling

In [4]:
mid_locs = ['420', '443', '490', '555', '638', '672', '702', '742', '782', '865', '910']
band_widths = [25, 40, 20, 20, 24, 16, 24, 16, 16, 20, 20]  

In [5]:
def resample_at (spectrum, mid_locs, band_widths):
    
    #----- obtaining the sampling ilocations in indx----
    mid_ilocs = []
    for l in mid_locs:
        #print(l)
        i = spectrum.columns.get_loc(l)
        mid_ilocs.append(i)
    
    #print(mid_ilocs)
    #print(mid_ilocs)
    #print(mid_locs)   
    
    #----- finding initial shape for red_spectra ------
    red_spectra = spectrum.iloc[:, mid_ilocs].copy()
    #print(red_spectra.head(5))
    #print(len(mid_ilocs))
    
    for i in range(0, len(mid_ilocs)):
        m = mid_ilocs[i]
        l = mid_locs[i]
        window = np.floor(0.5*band_widths[i]).astype(int)
        #print('m: ', m, 'window:', window)
        red_spectra.loc[:,l] = spectrum.iloc[:,(m-window):(m+window)].mean(axis=1).copy()
        
    return (red_spectra)


In [6]:
sampled_spec11 = resample_at (spec2[51], mid_locs, band_widths)
sampled_spec11.head(5)

,420,443,490,555,638,672,702,742,782,865,910
0,0.144063,0.166948,0.209295,0.292499,0.386718,0.407968,0.433550,0.462206,0.484446,0.506469,0.514470
1,0.119426,0.140218,0.178285,0.252382,0.330722,0.351590,0.375340,0.403457,0.426307,0.449363,0.457361
2,0.098955,0.120106,0.156299,0.228972,0.292581,0.305398,0.321836,0.341828,0.358382,0.372124,0.376753
3,0.179440,0.205586,0.248111,0.334626,0.410647,0.423945,0.442194,0.462696,0.478491,0.489124,0.492481
4,0.057449,0.066915,0.089151,0.129092,0.189341,0.211225,0.246785,0.293607,0.337743,0.413081,0.444682


In [7]:
sampled_cr11 = resample_at (cr_spec, mid_locs, band_widths)
#sampled_cr11.head(5)

In [8]:
fod_sampled11 = fod (sampled_spec11)
#fod_sampled13.head(5)

In [9]:
sampled_log11 = resample_at(log_spec, mid_locs, band_widths)
#sampled_log13.head(5)

In [10]:
import os
os.system('say "your Meta Tree started building"')
import time

In [11]:
# prepare_spec = ['none', 'fod2', 'continuum', 'log']
# prepare_target = ['none', 'minmax']

def find_X(p):
    if p == 'fod2':
        X = fod_sampled11
    elif p == 'cr':
        X = sampled_cr11
    elif p == 'log':
        X = sampled_log11
    else:
        X = sampled_spec11
    return X

def find_spec(p, m):    
    spec = find_X(p)
    return spec    


def find_y(t):
    i = target_names.index(t) 
    y = T[i]
    return y
        
        

## Recording Hypertuning Parameters

In [12]:
from sklearn.metrics import make_scorer, r2_score

model ={}
param_grid ={}
model['mult'] = LinearRegression()
param_grid['mult'] = { 'fit_intercept': [True, False] }

model['ridge'] = KernelRidge()
param_grid['ridge']={'alpha': [ 0.00001,0.00002, 0.00005, 0.0001, 0.0002, 0.0005, 0.001, 0.002, \
                               0.005,  0.01, 0.05, 0.1, 0.5, 1]}

model['plsr'] = PLSRegression()
param_grid['plsr']={'n_components': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}


model['cubist'] = Cubist(random_state = 42)  
param_grid['cubist'] = { 'n_committees': [5, 10, 15, 20], 'n_rules': [10, 20, 30, 40, 50] }
                        

model['gbrt'] = GradientBoostingRegressor(random_state = 42)
param_grid['gbrt'] = {
    'n_estimators': [30,40,50],        # Number of boosting stages to be used
    'learning_rate': [0.01, 0.1],     # Step size shrinkage used in updating weights
    'max_depth': [3, 4, 5]                # Maximum depth of individual trees
#     'min_samples_split': [2, 3],        # Minimum number of samples required to split an internal node
#     'min_samples_leaf': [1, 2],         # Minimum number of samples required to be at a leaf node
}

model['svr'] = SVR()
param_grid['svr'] = {
    'C': [0.1, 1, 10],              # Regularization parameter
    'kernel': ['linear', 'rbf'],   # Kernel function
    'gamma': [0.01, 0.1, 1],      # Kernel coefficient (only for 'rbf' kernel)
}


### Finding Hypertuned Parameters

In [13]:
# def find_core_model_param_on (m, t, p, tp, n):
#     Model = model[m]
#     X = find_spec(p,n,m)
#     y = find_y(t, tp)
    
#     row, col = X.shape

#     scorer = make_scorer(mean_squared_error, greater_is_better=False)
#     cv = RepeatedKFold(n_splits=row, n_repeats=1, random_state=3)
        
#     grid_search = GridSearchCV(Model, param_grid=param_grid[m], cv=cv,\
#                                 scoring= scorer, refit=True, verbose=1, error_score='raise', n_jobs=-1)
#     grid_search.fit(X,y)
    
#     return (grid_search.best_params_)

def find_tuned_param_on (X_train, y_train, m):
    Model = model[m]
    X = X_train
    y = y_train
    
    row, col = X.shape

    scorer = make_scorer(mean_squared_error, greater_is_better=False)
    cv = RepeatedKFold(n_splits=row, n_repeats=1, random_state=3)
        
    grid_search = GridSearchCV(Model, param_grid=param_grid[m], cv=cv,\
                                scoring= scorer, refit=True, verbose=1, error_score='raise', n_jobs=-1)
    grid_search.fit(X,y)
    
    return (grid_search.best_params_)
    

### Creating Hypertuned Models

In [14]:
def create_core_model_on (m, BP):
    if m == 'mult':
        Model = LinearRegression()
        
    elif m == 'ridge':
        Model =  KernelRidge(alpha = BP['alpha'])
        
    elif m == 'plsr':
        Model = PLSRegression(n_components = BP['n_components'])
        
    elif m == 'cubist':
        Model =  Cubist( n_committees = BP['n_committees'], n_rules = BP['n_rules'], random_state = 42) 
    
    elif m == 'gbrt':
        Model = GradientBoostingRegressor(n_estimators = BP['n_estimators'], learning_rate = BP['learning_rate'], \
                                          max_depth = BP['max_depth'],  random_state = 42)
    elif m == 'svr':
        Model = SVR(C = BP['C'], kernel = BP['kernel'], gamma = BP['gamma'])
        
    return Model    
            

## Building HMtree and BPtree

In [15]:
ml_methods = ['mult', 'ridge', 'plsr', 'svr', 'cubist', 'gbrt']

In [16]:
def L1_y_pred (spectra, target, method, bp):
    m = method
    (r,c) = spectra.shape
    
    Y_test = target.copy()
    Y_pred = target.copy()
    #print('r:', r)
    for i in range (0, r): 
        #print('i: ', i)
        full_spec = spectra.copy()
        X_train = full_spec.drop(full_spec.index[i], axis=0)
        X_test = full_spec.iloc[[i],:].copy()
        full_tar = target.copy()
        y_train = full_tar.drop(full_tar.index[i], axis=0)
        y_test = full_tar.iloc[i].copy()        
        #---- Model Creation, fitting, and predictions--------
        Model = create_core_model_on (m, bp)
        Model.fit(X_train, y_train)
        y_pred = Model.predict(X_test)
        Y_pred.iloc[i] = y_pred
        
    return Y_pred      

In [17]:
def build_HMtree_on (method_name, target):
    
    tree ={}
    
    start = time.time()
    m = method_name
    t = target
    
    print('tree for: '+ m +' ------> running on: ' + t)
    
    for tp in prepare_target:
        tree[tp] ={}        
        for p in prepare_spec:
            tree[tp][p] ={}            
            
            Y = tree[tp][p]
                                           
            #------ setting spec to appropriate (sampled) spectra----                
            spec = find_spec(p, m)                
                                        
            #---- target selection and normalization ---
            y = find_y(t)
                                
            #---- performing train-test split----------------------
            X_train, X_test, y_train, y_test = train_test_split(spec, y, test_size= 0.3, random_state=42)
                            
            #----- hypertuning parameter, model creation, fitting and prediction ----                
                
            #bp = find_tuned_param_on (X_train, y_train, m)
            bp = find_tuned_param_on (spec, y, m)
                
            Y['bp'] = bp
            print('Method: '+ m + '---> Target: '+ t+ '----> SpecProc: ' +p+ ' ---> n_band: 11')
            Model = create_core_model_on (m, bp)
            Model.fit(X_train, y_train)
                
            y_pred = Model.predict(X_test)
            yhat_pred = Model.predict(X_train)
                    
            #----- Data for Model accuracy and plotting -----------
            Y['X_test'] = X_test                
            Y['y_test'] = y_test
            Y['y_testP'] = y_pred
                
            Y['X_train'] = X_train
            Y['y_train'] = y_train
            Y['y_trainP'] = yhat_pred
                    
            Y['iqrp_test'] = find_iqrp(y_pred, y_test)
            Y['r2_test'] = find_r2(y_pred, y_test)
            Y['rpd_test'] = find_rpd(y_pred, y_test)                 
                
            Y['r2_train'] = find_r2(yhat_pred, y_train)
                
            #------- L1 out prediction on test data -----------------
            L1y_pred = L1_y_pred (spec, y, m, bp)
            Y['L1y_testP'] = L1y_pred
            Y['L1y_test'] = y
             
            
            Y['L1iqrp_test'] = find_iqrp(L1y_pred, y)
            Y['L1r2_test'] = find_r2(L1y_pred, y)
            Y['L1rpd_test'] = find_rpd(L1y_pred, y) 
                
                                                                         
    end = time.time()                            
    os.system('say "your hyper tuned meta tree finished for one more method"')                            

    print('End time - Start time =', (end-start)) 
    
    return tree.copy()

In [18]:
def build_HMtree_for (method_name):
    tree ={}
    
    start = time.time()
    m = method_name
    for t in target_names:
        #print('tree for: '+ m +' ------> running on: ' + t)
        tree[t] = build_HMtree_on (m, t)
        
    end = time.time()                            
    os.system('say "your meta tree finished for one more method"')                            
    print('End time - Start time =', (end-start))     
    return tree.copy()

In [19]:
def best_score_on (HMtree, target, method, scorer):
    t=target
    m= method
    
    best_score = -1
    #best_n_comp = 'NA'
    
    
    for tp in prepare_target:
        for p in prepare_spec:
            
            Y = HMtree[m][t][tp][p]
            #r2_train = Y['r2_train']
            #r2_test = Y['r2_test']
            #L1r2_test = Y['L1r2_test']
                 
                    
            if scorer == 'iqrp':
                cur_score = Y['iqrp_test']
            elif scorer == 'L1iqrp':
                cur_score = Y['L1iqrp_test']
            elif scorer == 'r2':
                cur_score = Y['r2_test'] 
            else:
                cur_score = Y['L1r2_test']  
                    
            if cur_score > best_score:
                best_score = cur_score
                best_tp = tp
                best_n = 13
                best_p = p
                            
    param_list = [scorer, np.round(best_score,2), 'Spec:', best_p, 'bands: 11']                                 
    return (param_list)          

In [20]:
def best_score_for (HMtree, target, scorer):
    
    for method in ml_methods:
        param_list= best_score_on (HMtree, target, method, scorer)
        print('For:'+target+'->', param_list, ':'+method)
     
    return

In [21]:
with open ('HMtreeVen.pickle', 'rb') as file:
    HMtree = pickle.load(file)
 

## Creating different branches of Mtree (for separate methods)

In [22]:
#HMtree = {}

In [23]:
# HMtree['mult'] = build_HMtree_for('mult')
# HMtree['plsr'] = build_HMtree_for('plsr')
# HMtree['svr'] = build_HMtree_for('svr')
# HMtree['ridge'] = build_HMtree_for('ridge')
# HMtree['cubist'] = build_HMtree_for('cubist')
# HMtree['gbrt'] = build_HMtree_for('gbrt')

tree for: mult ------> running on: Sand
Fitting 294 folds for each of 2 candidates, totalling 588 fits
Method: mult---> Target: Sand----> SpecProc: none ---> n_band: 11
Fitting 294 folds for each of 2 candidates, totalling 588 fits
Method: mult---> Target: Sand----> SpecProc: fod2 ---> n_band: 11
Fitting 294 folds for each of 2 candidates, totalling 588 fits
Method: mult---> Target: Sand----> SpecProc: cr ---> n_band: 11
Fitting 294 folds for each of 2 candidates, totalling 588 fits
Method: mult---> Target: Sand----> SpecProc: log ---> n_band: 11
End time - Start time = 47.41159772872925
tree for: mult ------> running on: Silt
Fitting 294 folds for each of 2 candidates, totalling 588 fits
Method: mult---> Target: Silt----> SpecProc: none ---> n_band: 11
Fitting 294 folds for each of 2 candidates, totalling 588 fits
Method: mult---> Target: Silt----> SpecProc: fod2 ---> n_band: 11
Fitting 294 folds for each of 2 candidates, totalling 588 fits
Method: mult---> Target: Silt----> SpecProc:

Fitting 294 folds for each of 14 candidates, totalling 4116 fits
Method: ridge---> Target: Silt----> SpecProc: cr ---> n_band: 11
Fitting 294 folds for each of 14 candidates, totalling 4116 fits
Method: ridge---> Target: Silt----> SpecProc: log ---> n_band: 11
End time - Start time = 92.25027394294739
tree for: ridge ------> running on: Clay
Fitting 294 folds for each of 14 candidates, totalling 4116 fits
Method: ridge---> Target: Clay----> SpecProc: none ---> n_band: 11
Fitting 294 folds for each of 14 candidates, totalling 4116 fits
Method: ridge---> Target: Clay----> SpecProc: fod2 ---> n_band: 11
Fitting 294 folds for each of 14 candidates, totalling 4116 fits
Method: ridge---> Target: Clay----> SpecProc: cr ---> n_band: 11
Fitting 294 folds for each of 14 candidates, totalling 4116 fits
Method: ridge---> Target: Clay----> SpecProc: log ---> n_band: 11
End time - Start time = 94.77086305618286
tree for: ridge ------> running on: TOC
Fitting 294 folds for each of 14 candidates, tota

C:\Users\abhis\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Method: cubist---> Target: Silt----> SpecProc: none ---> n_band: 11
Fitting 294 folds for each of 20 candidates, totalling 5880 fits
Method: cubist---> Target: Silt----> SpecProc: fod2 ---> n_band: 11
Fitting 294 folds for each of 20 candidates, totalling 5880 fits
Method: cubist---> Target: Silt----> SpecProc: cr ---> n_band: 11
Fitting 294 folds for each of 20 candidates, totalling 5880 fits
Method: cubist---> Target: Silt----> SpecProc: log ---> n_band: 11
End time - Start time = 1423.2792780399323
tree for: cubist ------> running on: Clay
Fitting 294 folds for each of 20 candidates, totalling 5880 fits


C:\Users\abhis\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Method: cubist---> Target: Clay----> SpecProc: none ---> n_band: 11
Fitting 294 folds for each of 20 candidates, totalling 5880 fits


C:\Users\abhis\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Method: cubist---> Target: Clay----> SpecProc: fod2 ---> n_band: 11
Fitting 294 folds for each of 20 candidates, totalling 5880 fits
Method: cubist---> Target: Clay----> SpecProc: cr ---> n_band: 11
Fitting 294 folds for each of 20 candidates, totalling 5880 fits
Method: cubist---> Target: Clay----> SpecProc: log ---> n_band: 11
End time - Start time = 4339.641870737076
tree for: cubist ------> running on: TOC
Fitting 294 folds for each of 20 candidates, totalling 5880 fits
Method: cubist---> Target: TOC----> SpecProc: none ---> n_band: 11
Fitting 294 folds for each of 20 candidates, totalling 5880 fits
Method: cubist---> Target: TOC----> SpecProc: fod2 ---> n_band: 11
Fitting 294 folds for each of 20 candidates, totalling 5880 fits
Method: cubist---> Target: TOC----> SpecProc: cr ---> n_band: 11
Fitting 294 folds for each of 20 candidates, totalling 5880 fits


C:\Users\abhis\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Method: cubist---> Target: TOC----> SpecProc: log ---> n_band: 11
End time - Start time = 4269.585121631622
End time - Start time = 11501.701481342316
tree for: gbrt ------> running on: Sand
Fitting 294 folds for each of 18 candidates, totalling 5292 fits
Method: gbrt---> Target: Sand----> SpecProc: none ---> n_band: 11
Fitting 294 folds for each of 18 candidates, totalling 5292 fits
Method: gbrt---> Target: Sand----> SpecProc: fod2 ---> n_band: 11
Fitting 294 folds for each of 18 candidates, totalling 5292 fits
Method: gbrt---> Target: Sand----> SpecProc: cr ---> n_band: 11
Fitting 294 folds for each of 18 candidates, totalling 5292 fits
Method: gbrt---> Target: Sand----> SpecProc: log ---> n_band: 11
End time - Start time = 498.83856439590454
tree for: gbrt ------> running on: Silt
Fitting 294 folds for each of 18 candidates, totalling 5292 fits
Method: gbrt---> Target: Silt----> SpecProc: none ---> n_band: 11
Fitting 294 folds for each of 18 candidates, totalling 5292 fits
Method: g

In [24]:
with open ('HMtreeVen.pickle', 'wb') as file:
    pickle.dump(HMtree, file)
        

In [25]:
#Plotting Model Accuracy (ipywidgets)
def plot_model_acc (method, target, spec_preprocessing):
    
    m = method
    t = target
    
    i = target_names.index(target)    
    
    p = spec_preprocessing
    #n = n_bands
    tp = 'none'
    
    Y = HMtree[m][t][tp][p]
    
    y_test = Y['y_test']
    y_pred = Y['y_testP']
    L1y_pred = Y['L1y_testP']
    L1y_test = Y['L1y_test']
    

    
    if m == 'plsr':
        y_pred = y_pred[:,0]
        #L1y_pred = L1y_pred[:,0]

    
    iqrp_test = Y['iqrp_test']
    r2_test = Y['r2_test']
    rpd_test = Y['rpd_test']
    
    L1iqrp_test = Y['L1iqrp_test']
    L1r2_test = Y['L1r2_test']
    L1rpd_test = Y['L1rpd_test']
    
    y_tp = pd.DataFrame({'actual':y_test.values, 'predic': y_pred})
    z = np.polyfit(y_test, y_pred, 1)
    
    L1y_tp = pd.DataFrame({'actual':L1y_test.values, 'predic': L1y_pred})
    L1z = np.polyfit(L1y_test, L1y_pred, 1)
    
    fig, axes = plt.subplots(2,2, figsize=(18,16))
    
    #with plt.style.context(('ggplot')): ---- PLOT of test-prediction --------------------------------------
    y_tp.plot.scatter(ax= axes[0][0], x="actual", y="predic", alpha=0.8, color = clr[i], edgecolors='k')
    axes[0][0].plot(y_test, np.polyval(z, y_test),  c='blue', linewidth=1)
    axes[0][0].plot(y_test, y_test, color='green', linewidth=1)
    axes[0][0].tick_params(axis='both', labelsize=10)
    axes[0][0].text(0.05, 0.95, target_names[i]+' (Train-Test Score)', transform=axes[0][0].transAxes, fontsize = 20, color = clr[i])
    axes[0][0].text(0.05, 0.90, 'IQRP ={:.2f}'.format(iqrp_test), transform=axes[0][0].transAxes, fontsize = 16)
    axes[0][0].text(0.05, 0.85, 'RPD ={:.2f}'.format(rpd_test), transform=axes[0][0].transAxes, fontsize = 16)
    axes[0][0].text(0.05, 0.80, 'R2 ={:.2f}'.format(np.round(r2_test,3)), transform=axes[0][0].transAxes, fontsize = 16)
    axes[0][0].text(0.95, 0.15, 'Method: '+method, transform=axes[0][0].transAxes, 
                    horizontalalignment='right', fontsize = 20)
    
    L1y_tp.plot.scatter(ax= axes[0][1], x="actual", y="predic", alpha=0.8, color = clr[i], edgecolors='k')
    axes[0][1].plot(L1y_test, np.polyval(L1z, L1y_test),  c='blue', linewidth=1)
    axes[0][1].plot(L1y_test, L1y_test, color='green', linewidth=1)
    axes[0][1].tick_params(axis='both', labelsize=10)
    axes[0][1].text(0.05, 0.95, target_names[i]+' (L1-out Score)', transform=axes[0][1].transAxes, fontsize = 20, color = clr[i])
    axes[0][1].text(0.05, 0.90, 'IQRP ={:.2f}'.format(L1iqrp_test), transform=axes[0][1].transAxes, fontsize = 16)
    axes[0][1].text(0.05, 0.85, 'RPD ={:.2f}'.format(L1rpd_test), transform=axes[0][1].transAxes, fontsize = 16)
    axes[0][1].text(0.05, 0.80, 'R2 ={:.2f}'.format(np.round(L1r2_test,3)), transform=axes[0][1].transAxes, fontsize = 16)
    axes[0][1].text(0.95, 0.15, 'Method: '+method, transform=axes[0][1].transAxes, 
                    horizontalalignment='right', fontsize = 20)

    
     #------------------------ Ploting R2 Vs N_bands-----------------------------------
    
    #--- data for creating n_bands vs r2 scores plot -------------------------------
    
#     pY = HMtree[m][t]['none'][p]        
#     X = nbands_sampling
    
#     Yr2 = []
#     L1Yr2 = []
    
#     for j in X:
#         Yr2.append(pY[j]['r2_test'])
#         L1Yr2.append(pY[j]['L1r2_test'])  
   
#     j = 0
#     while j < len(Yr2): 
#         if Yr2[j] <= 0:
#             Yr2[j] = 0
#         if L1Yr2[j] <= 0:
#             L1Yr2[j] = 0
#         j = j + 1
 
    #------------------------------ STEM PLOT ---- for accuracy Vs n_bands ---------------------------------- 
    
#     axes[1][0].stem(X,Yr2)
#     axes[1][0].tick_params(axis='both', labelsize=10)
#     axes[1][0].text(0.00, 1.01,  target_names[i], transform=axes[1][0].transAxes,fontsize = 20, color = clr[i])
#     axes[1][0].text(0.99, 1.01, 'Spec_prep: '+ p, transform=axes[1][0].transAxes, horizontalalignment='right', fontsize = 16)
#     axes[1][0].text(0.17, 1.01, '(train-test)', transform=axes[1][0].transAxes, horizontalalignment='left', fontsize = 16)
#     axes[1][0].text(0.60, -0.1, 'n_bands', transform=axes[1][0].transAxes, horizontalalignment='right', fontsize = 16)
#     axes[1][0].text(-0.1, 0.5, 'R2 Scores', horizontalalignment='left', verticalalignment='center', \
#                 rotation='vertical', transform=axes[1][0].transAxes, fontsize = 16)
    
    
#     axes[1][1].stem(X,L1Yr2)
#     axes[1][1].tick_params(axis='both', labelsize=10)
#     axes[1][1].text(0.00, 1.01,  target_names[i], transform=axes[1][1].transAxes,fontsize = 20, color = clr[i])
#     axes[1][1].text(0.99, 1.01, 'Spec_prep: '+ p, transform=axes[1][1].transAxes, horizontalalignment='right', fontsize = 16)
#     axes[1][1].text(0.17, 1.01, '(L1-out)', transform=axes[1][1].transAxes, horizontalalignment='left', fontsize = 16)
#     axes[1][1].text(0.60, -0.1, 'n_bands', transform=axes[1][1].transAxes, horizontalalignment='right', fontsize = 16)
#     axes[1][1].text(-0.1, 0.5, 'R2 Scores', horizontalalignment='left', verticalalignment='center', \
#                 rotation='vertical', transform=axes[1][1].transAxes, fontsize = 16)
        
   
    return    

In [26]:
#ml_methods = ['mult', 'ridge', 'plsr', 'svr', 'cubist', 'gbrt']

ml_methods = ['mult', 'plsr', 'svr', 'ridge', 'cubist','gbrt']

In [27]:
#best_score_for (HMtree, 'Sand', 'r2')

In [28]:
best_score_for (HMtree, 'Sand', 'L1r2')

For:Sand-> ['L1r2', 0.22, 'Spec:', 'none', 'bands: 11'] :mult
For:Sand-> ['L1r2', 0.25, 'Spec:', 'none', 'bands: 11'] :plsr
For:Sand-> ['L1r2', 0.17, 'Spec:', 'log', 'bands: 11'] :svr
For:Sand-> ['L1r2', -0.14, 'Spec:', 'fod2', 'bands: 11'] :ridge
For:Sand-> ['L1r2', 0.31, 'Spec:', 'none', 'bands: 11'] :cubist
For:Sand-> ['L1r2', 0.32, 'Spec:', 'fod2', 'bands: 11'] :gbrt


In [29]:
#best_score_for (HMtree, 'Silt', 'r2')

In [30]:
best_score_for (HMtree, 'Silt', 'L1r2')

For:Silt-> ['L1r2', 0.16, 'Spec:', 'fod2', 'bands: 11'] :mult
For:Silt-> ['L1r2', 0.19, 'Spec:', 'none', 'bands: 11'] :plsr
For:Silt-> ['L1r2', 0.09, 'Spec:', 'log', 'bands: 11'] :svr
For:Silt-> ['L1r2', 0.18, 'Spec:', 'log', 'bands: 11'] :ridge
For:Silt-> ['L1r2', 0.21, 'Spec:', 'fod2', 'bands: 11'] :cubist
For:Silt-> ['L1r2', 0.06, 'Spec:', 'log', 'bands: 11'] :gbrt


In [31]:
#best_score_for (HMtree, 'Clay', 'r2')

In [32]:
best_score_for (HMtree, 'Clay', 'L1r2')

For:Clay-> ['L1r2', 0.25, 'Spec:', 'log', 'bands: 11'] :mult
For:Clay-> ['L1r2', 0.25, 'Spec:', 'log', 'bands: 11'] :plsr
For:Clay-> ['L1r2', -0.08, 'Spec:', 'log', 'bands: 11'] :svr
For:Clay-> ['L1r2', 0.26, 'Spec:', 'log', 'bands: 11'] :ridge
For:Clay-> ['L1r2', 0.25, 'Spec:', 'log', 'bands: 11'] :cubist
For:Clay-> ['L1r2', 0.22, 'Spec:', 'fod2', 'bands: 11'] :gbrt


In [33]:
#best_score_for (HMtree, 'TOC', 'r2')

In [34]:
best_score_for (HMtree, 'TOC', 'L1r2')

For:TOC-> ['L1r2', 0.31, 'Spec:', 'fod2', 'bands: 11'] :mult
For:TOC-> ['L1r2', 0.38, 'Spec:', 'log', 'bands: 11'] :plsr
For:TOC-> ['L1r2', 0.42, 'Spec:', 'log', 'bands: 11'] :svr
For:TOC-> ['L1r2', 0.38, 'Spec:', 'log', 'bands: 11'] :ridge
For:TOC-> ['L1r2', 0.35, 'Spec:', 'fod2', 'bands: 11'] :cubist
For:TOC-> ['L1r2', 0.25, 'Spec:', 'fod2', 'bands: 11'] :gbrt


## Plotting Model Accuracy (ipywidgets)

In [35]:
ipywidgets.interact( plot_model_acc, target = target_names, method = ml_methods, \
                    spec_preprocessing = prepare_spec,  \
                    target_preprocessing = prepare_target)

interactive(children=(Dropdown(description='method', options=('mult', 'plsr', 'svr', 'ridge', 'cubist', 'gbrt'…

<function __main__.plot_model_acc(method, target, spec_preprocessing)>

In [36]:
with open ('HMtree.pickle', 'wb') as file:
    pickle.dump(HMtree, file)

In [37]:
# MTable = {'Input': ['Hyperspectral', 'Sentinal', 'Venus'], 
#           'Target': ['Sand', 'Sand', 'Sand'], 
#           'plsr': ['R2:0.57  bands:13','R2:0.57 bands:13','R2:0.57  bands:13'],
#           'cubist': ['R2:0.57  bands:13','R2:0.57 bands:13','R2:0.57  bands:13'],
#           'gbrt': ['R2:0.57  bands:13','R2:0.57 bands:13','R2:0.57  bands:13'],
#           'svr': ['R2:0.57  bands:13','R2:0.57 bands:13','R2:0.57  bands:13'],
#           'kridge': ['R2:0.57  bands:13','R2:0.57 bands:13','R2:0.57  bands:13']}
          

In [38]:
#MTable = pd.DataFrame.from_dict(MTable)


In [39]:
#pd.set_option('display.colheader_justify', 'center')

In [40]:
#MTable